## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-01-13-11 +0000,nypost,Ryder Cup fans as flat as US players’ game — a...,https://nypost.com/2025/09/27/sports/ryder-cup...
1,2025-09-28-01-06-43 +0000,nyt,Inside the Trump Administration’s Push to Pros...,https://www.nytimes.com/2025/09/27/us/politics...
2,2025-09-28-01-00-00 +0000,wsj,Xi Is Chasing a Huge Concession From Trump: Op...,https://www.wsj.com/world/china/trump-xi-talks...
3,2025-09-28-01-00-00 +0000,wsj,The Rise of America’s Young Socialists—From th...,https://www.wsj.com/us-news/socialists-zohran-...
4,2025-09-28-01-00-00 +0000,wsj,"Bob Iger, Hollywood’s Statesman, Gets a Politi...",https://www.wsj.com/business/media/bob-iger-ho...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2327/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
11,trump,45
633,president,8
12,administration,8
506,up,8
49,new,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
183,2025-09-27-02-03-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...,92
165,2025-09-27-06-45-39 +0000,nypost,US to revoke Colombian president’s visa after ...,https://nypost.com/2025/09/27/us-news/us-to-re...,90
100,2025-09-27-14-19-43 +0000,cbc,Trump administration asks U.S. Supreme Court t...,https://www.cbc.ca/news/world/trump-supreme-co...,79
91,2025-09-27-15-38-00 +0000,wsj,Trump Administration Asks Supreme Court to Uph...,https://www.wsj.com/us-news/law/trump-administ...,79
15,2025-09-27-23-18-15 +0000,nyt,"Trump Says He Has Ordered Troops to Portland, ...",https://www.nytimes.com/2025/09/27/us/politics...,79


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
183,92,2025-09-27-02-03-00 +0000,wsj,The Supreme Court on Friday granted the Trump ...,https://www.wsj.com/politics/policy/supreme-co...
165,45,2025-09-27-06-45-39 +0000,nypost,US to revoke Colombian president’s visa after ...,https://nypost.com/2025/09/27/us-news/us-to-re...
185,45,2025-09-27-02-00-00 +0000,wsj,A U.S. plan to field thousands of cutting-edge...,https://www.wsj.com/politics/national-security...
37,31,2025-09-27-21-27-07 +0000,nypost,Anti-ICE protesters led by paid activists marc...,https://nypost.com/2025/09/27/us-news/brooklyn...
146,28,2025-09-27-09-30-00 +0000,wsj,A new barrier for foreign workers could open s...,https://www.wsj.com/economy/jobs/trump-h1b-fee...
81,27,2025-09-27-17-01-42 +0000,nypost,Trump rages at bombshell report alleging FBI h...,https://nypost.com/2025/09/27/us-news/trump-ra...
87,25,2025-09-27-16-35-58 +0000,latimes,D4Vd's abandoned Tesla was ticketed 11 days be...,https://www.latimes.com/california/story/2025-...
63,25,2025-09-27-18-29-53 +0000,nypost,Israel strikes 120 targets in Gaza Strip — inc...,https://nypost.com/2025/09/27/world-news/israe...
186,23,2025-09-27-02-00-00 +0000,wsj,Kamala Harris’s book about the 2024 presidenti...,https://www.wsj.com/politics/kamala-harris-cam...
35,22,2025-09-27-21-38-32 +0000,nypost,"Erika Kirk seen smiling, makes announcement on...",https://nypost.com/2025/09/27/us-news/erika-ki...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
